In [ ]:
#<imports>
import os
from getpass import getpass

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.embeddings.fastembed import FastEmbedEmbedding
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI
from llama_index.core import Settings

from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
import chromadb

#<\imports>

In [ ]:
#<set hugging face api token>
HUGGINGFACEHUB_API_TOKEN = getpass("API:")
# Set the API token in the environment variable
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN
#<\set hugging face api token>

In [ ]:
#<load data>
documents = SimpleDirectoryReader("data").load_data()
#<\load data>

In [ ]:
#<set FastEmbeddings>
embed_model = FastEmbedEmbedding()
#<\set FastEmbeddings>

In [ ]:
#<set hugging face llm>
llm = HuggingFaceInferenceAPI(model_name="HuggingFaceH4/zephyr-7b-alpha", token=HUGGINGFACEHUB_API_TOKEN)
#<\set hugging face llm>

In [ ]:
#<By default LlamaIndex uses OpenAI, so we now override the settings>
Settings.llm = llm
Settings.embed_model = embed_model
#<By default LlamaIndex uses OpenAI, so we now override the settings>

In [ ]:
#<index the document>
db = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db.get_or_create_collection("quickstart")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex.from_documents(
documents, storage_context=storage_context, embed_model=embed_model)
#<\index the document>

In [ ]:
#<load the index>
db2 = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db2.get_or_create_collection("quickstart")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
index = VectorStoreIndex.from_vector_store(vector_store,embed_model=embed_model)
#<\load the index>

In [ ]:
#<defining query engine>
query_engine = index.as_query_engine()
#<\defining query engine>

In [ ]:
#<make a query>
response = query_engine.query("what is the point of the story")
print(response.response)
#<\make a query>